In [ ]:
import numpy as np

In [ ]:
from sklearn import datasets
from sklearn import decomposition

iris = datasets.load_iris()
X = iris.data[50:150]
y = iris.target[50:150]
pca = decomposition.PCA(n_components=2)
pca.fit(X)
X = pca.transform(X)

In [ ]:
import matplotlib.pyplot as plt

def plot_svc_decision_function(model):
    ax = plt.gca()
    
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    x = np.linspace(xlim[0], xlim[1], 30)
    y = np.linspace(ylim[0], ylim[1], 30)
    Y, X = np.meshgrid(y, x)
    xy = np.vstack([X.ravel(), Y.ravel()]).T
    P = model.decision_function(xy).reshape(X.shape)
    
    ax.contour(X, Y, P, levels=[-1, 0, 1], linestyles=['dashed', 'solid', 'dashed'])
    
    ax.scatter(model.support_vectors_[:, 0], model.support_vectors_[:, 1])
    
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

In [ ]:
from sklearn.svm import SVC

In [ ]:
clf = SVC(C=1.0, kernel="rbf", gamma=0.1)
clf.fit(X, y)
plt.scatter(X[:, 0], X[:, 1], c=y)
plot_svc_decision_function(clf)

In [ ]:
clf = SVC(C=1.0, kernel="rbf", gamma=10.0)
clf.fit(X, y)
plt.scatter(X[:, 0], X[:, 1], c=y)
plot_svc_decision_function(clf)

In [ ]:
clf = SVC(C=0.1, kernel="rbf", gamma=1.0)
clf.fit(X, y)
plt.scatter(X[:, 0], X[:, 1], c=y)
plot_svc_decision_function(clf)

In [ ]:
clf = SVC(C=10, kernel="rbf", gamma=1.0)
clf.fit(X, y)
plt.scatter(X[:, 0], X[:, 1], c=y)
plot_svc_decision_function(clf)

In [ ]:
from sklearn.model_selection import KFold

K = 5

C = [0.1, 0.5, 1, 5, 10]
SIGMA = [0.1, 0.5, 1, 5, 10]

kf = KFold(n_splits=K)

train_accuracies = np.zeros((len(C) * len(SIGMA), K))
test_accuracies = np.zeros((len(C) * len(SIGMA), K))
clfs = np.empty(25, dtype=SVC)

i = 0

for c in C:
    for sigma in SIGMA:
        j = 0
        
        for train_index, test_index in kf.split(X):
            clf = SVC(C=c, kernel="rbf", gamma=sigma)
            
            clf.fit(X[train_index], y[train_index])
            
            train_accuracies[i][j] = clf.score(X[train_index], y[train_index])
            test_accuracies[i][j] = clf.score(X[test_index], y[test_index])
            
            clfs[i] = clf
            
            j += 1
        
        i += 1

train_mean_accuracies = train_accuracies.mean(axis=1)
test_mean_accuracies = test_accuracies.mean(axis=1)

best_train_clf = clfs[train_mean_accuracies.argmax()]
best_test_clf = clfs[test_mean_accuracies.argmax()]

plt.figure()
plt.scatter(X[:, 0], X[:, 1], c=y)
plot_svc_decision_function(best_train_clf)
plt.title(
    "C = " + str(best_train_clf.C) +
    ", sigma = " + str(best_train_clf.gamma) +
    ", whole dataset accuracy = " + str(best_train_clf.score(X, y))
)

plt.figure()
plt.scatter(X[:, 0], X[:, 1], c=y)
plot_svc_decision_function(best_test_clf)
plt.title(
    "C = " + str(best_test_clf.C) +
    ", sigma = " + str(best_test_clf.gamma) +
    ", whole dataset accuracy = " + str(best_test_clf.score(X, y))
)